In [21]:
from llama_index.core.agent.workflow import FunctionAgent,AgentWorkflow
from llama_index.llms.openai import OpenAI

In [3]:
from llama_index.core.workflow import Context

In [4]:
llm = OpenAI(model="gpt-3.5-turbo-1106", temperature=0)

In [5]:
def multiply(a, b):
    return a * b

def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

def divide(a, b):
    return a / b

In [6]:
workflow = FunctionAgent(
    llm=llm,
    tools = [multiply, add],
    system_prompt="You are a agent that can call functions to perform calculations.",)

In [7]:
ctx = Context(workflow)

In [9]:
response = await workflow.run(user_msg="Hi, my name is Laurie!", ctx=ctx)
print(response)

Hello Laurie! How can I assist you today?


In [10]:
response2 = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(response2)

Your name is Laurie! How can I assist you further, Laurie?


In [11]:
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

In [12]:
ctx_dict = ctx.to_dict(serializer=JsonSerializer())

In [13]:
restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

response3 = await workflow.run(user_msg="What's my name?", ctx=restored_ctx)

In [15]:
print(response3)

You just told me your name is Laurie!


In [16]:
response3 = await workflow.run(user_msg="What's my name?")
print(response3)

I'm sorry, I can't help with that.


In [27]:
async def set_name(ctx: Context, name:str):
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return "Name set to " + name

In [28]:
workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

In [29]:
ctx = Context(workflow)

# check if it knows a name before setting it
response = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(str(response))

Your name is Alice.


In [30]:
response2 = await workflow.run(user_msg="My name is Laurie", ctx=ctx)
print(str(response2))

Your name is now Laurie.


In [31]:
state = await ctx.get("state")
print("Name as stored in state: ", state["name"])

Name as stored in state:  Laurie


In [32]:
response2 = await workflow.run(user_msg="My name is Patrick", ctx=ctx)
print(str(response2))

Your name is now Patrick.


In [33]:
state = await ctx.get("state")
print("Name as stored in state: ", state["name"])

Name as stored in state:  Patrick
